## 1. Read the census data using pandas library

In [1]:
import pandas as pd

In [2]:
census = pd.read_csv('census_data.csv')

## 2. Display the head of the dataset

In [3]:
census.head()

,age,workclass,education,education_num,marital_status,occupation,relationship,race,gender,capital_gain,capital_loss,hours_per_week,native_country,income_bracket
0,39,State-gov,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


## 3. Convert the label column to 0s and 1s instead of strings

In [4]:
census['income_bracket'].unique()

array([' <=50K', ' >50K'], dtype=object)

In [5]:
def lable_fix(label):
    if label == ' <=50K':
        return 0
    else:
        return 1

In [6]:
census['income_bracket'] = census['income_bracket'].apply(lable_fix)

## 4. Perform the train test split on the data

In [7]:
from sklearn.model_selection import train_test_split

In [8]:
x_data = census.drop('income_bracket' , axis = 1)
y_lables = census['income_bracket']
x_train, x_test, y_train, y_test = train_test_split(x_data, y_lables, test_size=0.3, random_state=101)

In [9]:
census.columns

Index(['age', 'workclass', 'education', 'education_num', 'marital_status',
       'occupation', 'relationship', 'race', 'gender', 'capital_gain',
       'capital_loss', 'hours_per_week', 'native_country', 'income_bracket'],
      dtype='object')

## 5. Create feature columns for categorical values using vocabulary lists or hash buckets

In [11]:
import tensorflow as tf

In [12]:
gender = tf.feature_column.categorical_column_with_vocabulary_list("gender" , ["Female", "Male"])
occupation = tf.feature_column.categorical_column_with_hash_bucket("occupation" , hash_bucket_size=1000)
marital_status = tf.feature_column.categorical_column_with_hash_bucket("marital_status" , hash_bucket_size=1000)
relationship = tf.feature_column.categorical_column_with_hash_bucket("relationship", hash_bucket_size=1000)
education = tf.feature_column.categorical_column_with_hash_bucket("education" ,hash_bucket_size=1000)
workclass = tf.feature_column.categorical_column_with_hash_bucket("workclass", hash_bucket_size=1000)
native_country = tf.feature_column.categorical_column_with_hash_bucket("native_country", hash_bucket_size=1000)

## 6. Create the feature columns for the continuous values using numeric_column

In [13]:
age = tf.feature_column.numeric_column("age")
education_num = tf.feature_column.numeric_column("education_num")
capital_gain = tf.feature_column.numeric_column("capital_gain")
capital_loss = tf.feature_column.numeric_column("capital_loss")
hours_per_week = tf.feature_column.numeric_column("hours_per_week")

## 7. Put all these variables into a single list with variable name feat_cols

In [14]:
feat_cols = [gender, occupation, marital_status, relationship, education, workclass, native_country, age, education_num, 
             capital_gain, capital_loss, hours_per_week]

## 8. Create the input function with batch size and epochs

In [15]:
input_func = tf.estimator.inputs.pandas_input_fn(x=x_train, y=y_train, batch_size=100, num_epochs=None, shuffle=True)

## 9. Create the model with tf.estimator using Linear Classifier

In [16]:
model = tf.estimator.LinearClassifier(feature_columns=feat_cols)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\malic\\AppData\\Local\\Temp\\tmprfey3xgg', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000002174AE95C88>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


## 10. Train the model for atleast 5000 steps

In [17]:
model.train(input_fn=input_func, steps=5000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into C:\Users\malic\AppData\Local\Temp\tmprfey3xgg\model.ckpt.
INFO:tensorflow:loss = 69.31472, step = 1
INFO:tensorflow:global_step/sec: 76.409
INFO:tensorflow:loss = 358.03915, step = 101 (1.314 sec)
INFO:tensorflow:global_step/sec: 168.235
INFO:tensorflow:loss = 343.34198, step = 201 (0.593 sec)
INFO:tensorflow:global_step/sec: 176.585
INFO:tensorflow:loss = 727.407, step = 301 (0.568 sec)
INFO:tensorflow:global_step/sec: 173.558
INFO:tensorflow:loss = 754.88605, step = 401 (0.575 sec)
INFO:tensorflow:global_step/sec: 175.793
INFO:tensorflow:loss = 983.00995, step = 501 (0.570 sec)
INFO:tensorflow:global_step/sec: 176.785
INFO:tensorflow:loss = 175.8079, step = 601 (0.564 sec)
INFO:tensorflow:global_step/sec: 

## 11. Evalution of the model

In [18]:
pred_fn = tf.estimator.inputs.pandas_input_fn(x=x_test, batch_size=len(x_train), shuffle=False)

In [19]:
# Using model.predict() and passing the input function. 
# This will produce a generator of predictions, which can then transformed into list
predictions = list(model.predict(input_fn=pred_fn))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\malic\AppData\Local\Temp\tmprfey3xgg\model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [20]:
# Each item in the list will look like this
predictions[0]

{'logits': array([-0.75077933], dtype=float32),
 'logistic': array([0.32065153], dtype=float32),
 'probabilities': array([0.67934847, 0.3206515 ], dtype=float32),
 'class_ids': array([0], dtype=int64),
 'classes': array([b'0'], dtype=object)}

## 12. Create a list of class_ids key values from the prediction list of dictionaries. These prediction will be used to compare against y_test values 

In [21]:
final_preds = []
for pred in predictions:
    final_preds.append(pred['class_ids'][0])

In [22]:
final_preds[:10]

[0, 0, 0, 0, 1, 0, 0, 0, 0, 0]

## 13. Calculating the model performance on Test Data

In [23]:
from sklearn.metrics import classification_report

In [24]:
print(classification_report(y_test, final_preds))

             precision    recall  f1-score   support

          0       0.90      0.90      0.90      7436
          1       0.69      0.69      0.69      2333

avg / total       0.85      0.85      0.85      9769

